# Cuaderno de soporte para IL-CLDM
Este cuaderno concentra los comandos auxiliares que usamos para entrenar y diagnosticar el modelo de difusión latente condicionado (IL-CLDM).
- **Entrenamiento**: controla la ejecución del script principal y de cada etapa (AAE, codificador de latentes y difusor).
- **Diagnóstico**: agrupa rutinas de inspección para verificar datos, métricas y distribuciones latentes.
> Recomendación: revisa/ajusta `config.py` antes de ejecutar cualquier celda y avanza en orden según la sección que necesites.

## 1. Ejecución y entrenamiento principal
Las celdas siguientes lanzan el entrenamiento desde el notebook. Ejecuta solo las que necesites:
1. `!python main.py` corre el flujo completo desde la terminal (entrena AAE, codifica latentes y ajusta el LDM).
2. `main.train_AAE()` entrena únicamente el autoencoder adversario.
3. `main.encoding()` genera/actualiza los latentes SPECT tras entrenar el AAE.
4. `main.train_LDM()` entrena el modelo de difusión condicionado usando los latentes generados.

In [ ]:
# Configura variables de entorno compartidas para las ejecuciones posteriores.
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
# Ejecuta la tubería completa (AAE + codificación + LDM) desde la línea de comandos.
!python main.py

In [ ]:
# Permite invocar funciones específicas del pipeline directamente desde Python.
import main

In [ ]:
# Entrena solo la primera etapa (AAE) sin necesidad de correr todo `main.py`.
main.train_AAE()

In [ ]:
# Genera/actualiza los latentes SPECT una vez ajustado el AAE.
main.encoding()

In [ ]:
# Empieza el entrenamiento del modelo de difusión condicionado usando los latentes almacenados.
main.train_LDM()

## 2. Evaluación cuantitativa del AAE en SPECT
Usa estas celdas para comparar reconstrucciones con los datos reales mediante PSNR/SSIM y ejecutar `aae_eval.py` cuando necesites verificar la convergencia de la primera etapa antes de continuar.

In [ ]:
import os, nibabel as nib, numpy as np, torch
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import config
from model import AAE
from utils import load_checkpoint
from dataset import center_crop

device = torch.device(config.device)
out_dir = os.path.join("result", str(config.exp), "aae_recon_test")
os.makedirs(out_dir, exist_ok=True)

latent_dir = config.latent_Abeta
sample_id = os.listdir(latent_dir)[0]  # o elige uno
latent_path = os.path.join(latent_dir, sample_id)

lat = nib.load(latent_path).get_fdata().astype(np.float32)
x_lat = torch.tensor(lat[None,None,...], device=device)
aae = AAE().to(device)
opt = torch.optim.Adam(aae.parameters(), lr=config.learning_rate)
load_checkpoint(config.CHECKPOINT_AAE, aae, opt, config.learning_rate)
aae.eval()

with torch.no_grad():
    recon = aae.decoder(x_lat)
recon = torch.clamp(recon, 0, 1).cpu().numpy().squeeze().astype(np.float32)

orig_id = os.path.splitext(sample_id)[0]
orig_path = os.path.join(config.whole_Abeta, orig_id + ".nii")
if not os.path.exists(orig_path):
    orig_path = orig_path + "gz"  # si es .nii.gz
orig_img = nib.load(orig_path)
orig = center_crop(orig_img.get_fdata().astype(np.float32), config.crop_size)
vmin, vmax = orig.min(), orig.max()
orig = np.zeros_like(orig) if vmax - vmin < 1e-8 else (orig - vmin)/(vmax - vmin)
rng = max(orig.max() - orig.min(), 1e-8)
print("PSNR", psnr(orig, recon, data_range=rng), "SSIM", ssim(orig, recon, data_range=rng))

nib.save(nib.Nifti1Image(recon, orig_img.affine),
         os.path.join(out_dir, f"{orig_id}_decoder_only.nii.gz"))

In [ ]:
import numpy as np, nibabel as nib, csv, torch
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import config
from dataset import resolve_nifti, center_crop  # usa center_crop
from model import AAE

device = torch.device(config.device)
model = AAE().to(device)
opt = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
ckpt = torch.load(config.CHECKPOINT_AAE, map_location=device)
model.load_state_dict(ckpt["state_dict"])
model.eval()

ids = [i.strip() for i in open(config.test) if i.strip()]
psnr_sum = ssim_sum = 0

with open("result/exp_1/aae_recon_test.csv", "w", newline="") as f:
    writer = csv.writer(f); writer.writerow(["ID","PSNR","SSIM"])
    for bid in ids:
        path = resolve_nifti(config.whole_Abeta, bid)
        img = nib.load(path)
        data = img.get_fdata().astype(np.float32)
        data = center_crop(data, config.crop_size)
        vmin, vmax = data.min(), data.max()
        data = np.zeros_like(data) if vmax - vmin < 1e-8 else (data - vmin)/(vmax - vmin)

        x = torch.tensor(data[None,None,...], device=device)
        with torch.no_grad():
            recon = model(x)
        recon = torch.clamp(recon, 0, 1).cpu().numpy().squeeze().astype(np.float32)

        # Asegúrate de que coinciden las formas
        if recon.shape != data.shape:
            print(f"{bid} shapes mismatch: data {data.shape}, recon {recon.shape}")
            continue

        rng = max(data.max() - data.min(), 1e-8)
        ps = psnr(data, recon, data_range=rng)
        ss = ssim(data, recon, data_range=rng)
        writer.writerow([bid, ps, ss])
        psnr_sum += ps; ssim_sum += ss

print("Promedio PSNR", psnr_sum/len(ids), "SSIM", ssim_sum/len(ids))

In [ ]:
import pandas as pd

aae_results = pd.read_csv("/workspace/projects/T1-SPECT-translation/IL-CLDM/result/exp_1/aae_recon_test/aae_metrics.csv")
aae_results.head()

In [ ]:
print(aae_results["PSNR"].mean(), aae_results["PSNR"].std(), aae_results["SSIM"].mean(), aae_results["SSIM"].std())

In [ ]:
!python aae_eval.py

In [ ]:
import os, nibabel as nib, numpy as np, torch
gen_path = "/workspace/projects/T1-SPECT-translation/IL-CLDM/data/latent_SPECT"
files = os.listdir(gen_path)

for file in files:
    #getting the shape of each file
    latent = nib.load(os.path.join(gen_path, file)).get_fdata()
    if latent.shape != (40, 48, 40):
        print(f"File: {file}, Shape: {latent.shape}")

## 3. Exploración de la representación latente de MRI
Permite inspeccionar los archivos latentes generados por el encoder para confirmar dimensiones (`config.latent_shape`) y rangos antes de alimentar el difusor.

In [ ]:
import os, nibabel as nib, torch
from dataset import resolve_nifti, center_crop, z_score_norm
from model import UNet
import config
import numpy as np

device = torch.device(config.device)

# Carga el UNet con el image_size correcto
unet = UNet(in_channel=2, out_channel=1, image_size=config.latent_shape[0]).to(device)
unet.eval()

ids = [i.strip() for i in open(config.train) if i.strip()][:3]  # prueba con 3 IDs
out_dir = "result/exp_unet_sd/mri_latent_cond"
os.makedirs(out_dir, exist_ok=True)

with torch.no_grad():
    for bid in ids:
        mri_path = resolve_nifti(config.whole_MRI, bid)
        img = nib.load(mri_path)
        mri = img.get_fdata().astype(np.float32)
        # mismo preprocesado que en TwoDataset
        from dataset import resample_to_shape
        mri = resample_to_shape(mri, config.target_shape)
        mri = center_crop(mri, config.crop_size)
        mri = z_score_norm(mri)

        x = torch.tensor(mri[None, None, ...], device=device)
        cond_latent = unet.cond(x).cpu().numpy().squeeze()  # esperado: 40x48x40 con config actual

        print(bid, "cond shape", cond_latent.shape, "min/max", cond_latent.min(), cond_latent.max())
        nib.save(nib.Nifti1Image(cond_latent, img.affine),
                 os.path.join(out_dir, f"{bid}_mri_cond.nii.gz"))


## 4. Verificación de datos de entrada
Aquí puedes abrir volúmenes MRI/SPECT brutos, revisar sus minas/máximos y asegurarte de que la nomenclatura de archivos coincida con `data_info/`.

In [ ]:
import nibabel as nib
import numpy as np
import os

base_path = "/workspace/projects/T1-SPECT-translation/IL-CLDM/data/whole_SPECT"

files = os.listdir(base_path)
nan_values = []

for file in files:
    if file.endswith(".nii"):
        filepath = os.path.join(base_path, file)
        img = nib.load(filepath)
        data = img.get_fdata()
        print("dimansions of the file:", data.shape)
        if np.isnan(data).any():
            print(f"El archivo {file} contiene NaNs.")
            #cuantos NaNs hay
            num_nans = np.isnan(data).sum()
            print(f"Número de NaNs en el archivo {file}: {num_nans}")
            nan_values.append((file, num_nans))
        else:
            print(f"El archivo {file} no contiene NaNs.")

#obtener el file con menos NaNs y el de mas NaNs
if nan_values:
    nan_values.sort(key=lambda x: x[1])
    print(f"Archivo con menos NaNs: {nan_values[0][0]} con {nan_values[0][1]} NaNs")
    print(f"Archivo con más NaNs: {nan_values[-1][0]} con {nan_values[-1][1]} NaNs")

In [ ]:
file = "3863.nii"
filepath = os.path.join(base_path, file)
img = nib.load(filepath)
data = img.get_fdata()
print("dimansions of the file:", data.shape)

In [ ]:
import nibabel as nib, numpy as np
from pathlib import Path

def resolve(root, bid):
    for ext in ('.nii', '.nii.gz'):
        p = Path(root)/f"{bid}{ext}"
        if p.exists(): return p
    raise FileNotFoundError(bid)

def stats(path):
    data = nib.load(str(path)).get_fdata()
    data = np.nan_to_num(data, nan=0.0, posinf=0.0, neginf=0.0)
    return data.min(), data.max(), data.mean(), data.std()

root_abeta = Path('data/whole_SPECT')
root_mri = Path('data/whole_MRI')
train_ids = Path('data_info/train.txt').read_text().splitlines()[:-10]  # cambia [:3] por más
print('IDs', train_ids)
for bid in train_ids:
    pA = resolve(root_abeta, bid)
    pM = resolve(root_mri, bid)
    amin, amax, amean, astd = stats(pA)
    mmin, mmax, mmean, mstd = stats(pM)
    print(f"{bid} Abeta: min {amin:.4f} max {amax:.4f} mean {amean:.4f} std {astd:.4f}")
    print(f"{bid} MRI  : min {mmin:.4f} max {mmax:.4f} mean {mmean:.4f} std {mstd:.4f}")

## 5. Métricas locales PSNR/SSIM
Fragmentos rápidos para validar manualmente las métricas sobre casos concretos o ruido sintético, útiles cuando se ajustan ventanas o normalizaciones.

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from dataset import OneDataset
import config

ds = OneDataset(root_Abeta=config.whole_Abeta, task=config.validation, stage="validation")
for i in range(3):  # primeros 3
    vol, name = ds[i]
    vol = vol.astype(np.float32)
    rng = max(vol.max() - vol.min(), 1e-8)
    print(name, "range", vol.min(), vol.max(),
          "PSNR self", psnr(vol, vol, data_range=rng),
          "SSIM self", ssim(vol, vol, data_range=rng))
    # Volumen con ruido para ver caída
    noise = np.random.normal(0, 0.01, size=vol.shape).astype(np.float32)
    vol_noisy = np.clip(vol + noise, 0, 1)
    print("  noisy PSNR", psnr(vol, vol_noisy, data_range=rng),
          "noisy SSIM", ssim(vol, vol_noisy, data_range=rng))

In [ ]:
import os, nibabel as nib, numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import config

ids = open(config.validation).read().split()
for bid in ids[:10]:  # revisa los primeros 10
    path = None
    for ext in (".nii", ".nii.gz"):
        p = os.path.join(config.whole_Abeta, bid + ext)
        if os.path.exists(p):
            path = p
            break
    if not path:
        print("no file for", bid)
        continue
    img = nib.load(path).get_fdata().astype(np.float32)
    vol = img[10:170, 18:210, 10:170]  # mismo crop
    vmin, vmax = vol.min(), vol.max()
    vol = np.zeros_like(vol) if vmax - vmin < 1e-8 else (vol - vmin) / (vmax - vmin)
    rng = max(vol.max() - vol.min(), 1e-8)
    psnr_self = psnr(vol, vol, data_range=rng)
    ssim_self = ssim(vol, vol, data_range=rng)
    print(bid, "range", vol.min(), vol.max(), "psnr_self", psnr_self, "ssim_self", ssim_self)
    if np.isfinite(ssim_self) and ssim_self > 0.9:
        break


## 6. Curvas de entrenamiento y validación
Carga los CSV generados durante el entrenamiento (`loss_curve.csv`, `validation.csv`, etc.) para visualizar la evolución de pérdidas y métricas por época.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
gen_path = "/workspace/projects/T1-SPECT-translation/IL-CLDM/result/exp_unet_sd"
file1 = "loss_curve.csv"
file2 = "validation.csv"

loss_df = pd.read_csv(os.path.join(gen_path, file1))
similarity_df = pd.read_csv(os.path.join(gen_path, file2))

In [ ]:
loss_df.head()

In [ ]:
similarity_df.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = loss_df

# --- Convertir columnas a numérico (forzando errores a NaN) ---
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors="ignore")

# Si epoch, PSNR o SSIM están como strings, forzamos:
df["Epoch"] = pd.to_numeric(df["Epoch"], errors="coerce")
df["recon_loss"] = pd.to_numeric(df["recon_loss"], errors="coerce")
df["disc_loss_epoch"] = pd.to_numeric(df["disc_loss_epoch"], errors="coerce")

# Eliminar filas que quedaron con NaN en métricas o epochs
df = df.dropna(subset=["Epoch", "recon_loss", "disc_loss_epoch"])

# Redondear PSNR y SSIM a 2 decimales
df["recon_loss"] = df["recon_loss"].round(2)
df["disc_loss_epoch"] = df["disc_loss_epoch"].round(2)

# Crear figura
plt.figure(figsize=(10,5))

# Graficar PSNR
plt.plot(df["Epoch"], df["recon_loss"], marker='o', label="recon_loss", linewidth=2)

# Graficar SSIM
plt.plot(df["Epoch"], df["disc_loss_epoch"], marker='s', label="disc_loss_epoch", linewidth=2)

# Títulos y ejes
plt.title("recon_loss y disc_loss_epoch vs Épocas")
plt.xlabel("Épocas")
plt.ylabel("Valor Métrica")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend()

# Mostrar
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = similarity_df

# --- Convertir columnas a numérico (forzando errores a NaN) ---
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors="ignore")

# Si epoch, PSNR o SSIM están como strings, forzamos:
df["Epoch"] = pd.to_numeric(df["Epoch"], errors="coerce")
df["PSNR"] = pd.to_numeric(df["PSNR"], errors="coerce")
df["SSIM"] = pd.to_numeric(df["SSIM"]*100, errors="coerce")

# Eliminar filas que quedaron con NaN en métricas o epochs
df = df.dropna(subset=["Epoch", "PSNR", "SSIM"])

# Redondear PSNR y SSIM a 2 decimales
df["PSNR"] = df["PSNR"].round(2)
df["SSIM"] = df["SSIM"].round(2)

# Crear figura
plt.figure(figsize=(10,5))

# Graficar PSNR
plt.plot(df["Epoch"], df["PSNR"], marker='o', label="PSNR", linewidth=2)

# Graficar SSIM
plt.plot(df["Epoch"], df["SSIM"], marker='s', label="SSIM", linewidth=2)

# Títulos y ejes
plt.title("PSNR y SSIM vs Épocas")
plt.xlabel("Épocas")
plt.ylabel("Valor Métrica")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend()

# Mostrar
plt.show()

## 7. Conversión de .nii.gz a .nii
Utiliza estas celdas cuando necesites trabajar con herramientas que solo aceptan `.nii`, manteniendo una copia limpia en el directorio de datos.

In [ ]:
import nibabel as nib
import gzip
import shutil
from pathlib import Path

In [ ]:
input_folder = "/workspace/projects/T1-SPECT-translation/IL-CLDM/data/whole_SPECT"
output_folder = "/workspace/projects/T1-SPECT-translation/IL-CLDM/data/whole_SPECT2"
Path(output_folder).mkdir(parents=True, exist_ok=True)

for f in Path(input_folder).glob("*.nii.gz"):
    img = nib.load(str(f))
    out_path = Path(output_folder) / f.with_suffix('').name  # quita .gz → .nii
    nib.save(img, str(out_path))
    print("Guardado:", out_path)

## 8. Inferencia con el modelo entrenado
Lanza `infer.py` para generar volúmenes sintéticos usando los checkpoints actuales. Ideal para validar resultados cualitativos tras el entrenamiento.

In [ ]:
!python infer.py

## 9. Carga manual del UNet de difusión
Proporciona utilidades para inspeccionar la arquitectura, cargar pesos y depurar tensores del modelo de difusión acondicionado.

In [ ]:
import torch
from model import UNet
import config

device = torch.device(config.device)
unet = UNet(in_channel=2, out_channel=1, image_size=config.latent_shape[0]).to(device)
print(unet)  # lista todos los módulos, incluidas las capas con atención

In [ ]:
import torch
from torchinfo import summary
from model import UNet
import config

device = torch.device(config.device)
latent_shape = config.latent_shape       # p.ej. (40,48,40)
crop_shape   = config.crop_size          # p.ej. (160,192,160)

unet = UNet(in_channel=2, out_channel=1, image_size=latent_shape[0]).to(device)

x_dummy = torch.randn(1, 1, *latent_shape, device=device)   # latente SPECT
mri_dummy = torch.randn(1, 1, *crop_shape, device=device)    # MRI recortada
t_dummy = torch.tensor([10], device=device, dtype=torch.long)
label_dummy = torch.tensor([0], device=device, dtype=torch.long)  # o None si no usas label

summary(unet,
        input_data=(x_dummy, mri_dummy, t_dummy, label_dummy),
        depth=3)  # ajusta depth si quieres menos/más detalle

## 10. Estadísticas de los latentes SPECT y MRI
Reúne scripts para calcular histogramas, medias y desviaciones estándar tanto de los latentes reales como de los sintetizados.

In [ ]:
import os, nibabel as nib, numpy as np, config
import glob

files = glob.glob(os.path.join(config.latent_Abeta, "*.nii*"))  # muestrea 100
vals = []
for f in files:
    x = nib.load(f).get_fdata().astype(np.float32)
    vals.append(x.reshape(-1))
vals = np.concatenate(vals)
print("Latente SPECT shape esperado", config.latent_shape)
print("min", vals.min(), "max", vals.max(), "mean", vals.mean(), "std", vals.std())

### 10.1 Estadísticas del acondicionamiento MRI
Calcula estadísticas básicas de los tensores de acondicionamiento derivados del MRI para asegurarse de que comparten rango con los latentes SPECT.

In [ ]:
import torch, nibabel as nib, numpy as np, config
from dataset import resolve_nifti, resample_to_shape, center_crop, z_score_norm
from model import UNet

device = torch.device(config.device)
unet = UNet(in_channel=2, out_channel=1, image_size=config.latent_shape[0]).to(device)
unet.eval()

ids = [i.strip() for i in open(config.train) if i.strip()] 
vals = []
with torch.no_grad():
    for bid in ids:
        mri_path = resolve_nifti(config.whole_MRI, bid)
        mri = nib.load(mri_path).get_fdata().astype(np.float32)
        mri = resample_to_shape(mri, config.target_shape)
        mri = center_crop(mri, config.crop_size)
        mri = z_score_norm(mri)
        x = torch.tensor(mri[None,None,...], device=device)
        cond = unet.cond(x).cpu().numpy().squeeze()  # debería ser 40×48×40
        vals.append(cond.reshape(-1))
vals = np.concatenate(vals)
print("Cond MRI shape", cond.shape)
print("min", vals.min(), "max", vals.max(), "mean", vals.mean(), "std", vals.std())

### 10.2 Estadísticas de latentes sintetizados
Evalúa los latentes generados por el difusor (después de muestrear `ema_Unet`) para confirmar que mantienen la distribución objetivo antes de decodificar a espacio imagen.

In [ ]:
import os, glob, nibabel as nib, numpy as np, torch
import config
from model import AAE, UNet
from main import Diffusion
from dataset import resolve_nifti, resample_to_shape, center_crop, z_score_norm

device = torch.device(config.device)

# Carga modelos
aae = AAE().to(device)
opt = torch.optim.Adam(aae.parameters(), lr=config.learning_rate)
aae_ckpt = torch.load(config.CHECKPOINT_AAE, map_location=device)
aae.load_state_dict(aae_ckpt["state_dict"])
aae.eval()

unet = UNet(in_channel=2, out_channel=1, image_size=config.latent_shape[0]).to(device)
opt_u = torch.optim.AdamW(unet.parameters(), lr=config.learning_rate)
print("Cargando UNet desde", config.CHECKPOINT_Unet)
state = torch.load(config.CHECKPOINT_Unet, map_location=device)
sd = state.get("unet_state_dict") or state.get("state_dict")
# quitar prefijo module. si existe
sd = {k.replace("module.", "", 1): v for k, v in sd.items()}
unet.load_state_dict(sd, strict=False)
unet.eval()
diffusion = Diffusion()

# Elige algunas MRI
ids = [i.strip() for i in open(config.train) if i.strip()][:15]
latents = []
with torch.no_grad():
    for bid in ids:
        mri_path = resolve_nifti(config.whole_MRI, bid)
        mri = nib.load(mri_path).get_fdata().astype(np.float32)
        mri = resample_to_shape(mri, config.target_shape)
        mri = center_crop(mri, config.crop_size)
        mri = z_score_norm(mri)
        x = torch.tensor(mri[None,None,...], device=device)
        # sample latent from diffusion
        samp_lat = diffusion.sample(unet, x)
        latents.append(samp_lat.cpu().numpy().reshape(-1))

latents = np.concatenate(latents)
print("Latente difusión -> decoder:")
print("shape esperado", config.latent_shape, "min", latents.min(), "max", latents.max(), "mean", latents.mean(), "std", latents.std())

In [ ]:
latents = np.concatenate(latents)
print("Latente difusión -> decoder:")
print("shape esperado", config.latent_shape, "min", latents.min(), "max", latents.max(), "mean", latents.mean(), "std", latents.std())

## 11. Impacto del condicionamiento por etiqueta
Reserva estas celdas para experimentos donde se compare el entrenamiento con/ sin `label` como condición adicional en el UNet.

In [ ]:
# Añade aquí los experimentos cuantitativos sobre el condicionamiento de etiquetas cuando estén listos.
